In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
from pathlib import Path
# new_version_path = Path(r'D:\vnpy-2.0.8')
new_version_path = Path(r'E:\vnpy\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

import vnpy
print(vnpy.__version__)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional

from vnpy.app.cta_strategy.backtesting import BacktestingEngine
from vnpy.trader.constant import Offset

from utility import comodity_to_vt_symbol, get_output_path, vt_trade_to_df, trade_zh_to_en
from backtesting import ResearchBacktestingEngine
from trade_match import calculate_trades_result, generate_trade_df, exhaust_trade_result
from turtle_b_strategy import TurtleBStrategy

future_basic_data = pd.read_csv('future_basic_data.csv', index_col=0)
future_hot_start = pd.read_csv('future_hot_start.csv', index_col=0,  header=None, names=['hot_start'])

def get_hot_start(commodity: str) -> datetime:
    start = future_hot_start.loc[commodity.upper()]['hot_start']
    start = datetime.strptime(start, '%Y-%m-%d')
    if not pd.isna(start):
        return start
    
def params_to_str(params: dict) -> str:
    if params:
        return '.'.join([f"{k}_{v}" for k, v in params.items()])
    else:
        return 'default'

def run_research_backtest(
    commodity: str,
    start: datetime,
    end: datetime,
    trade_output = False,
    curve_output = False,
    strategy_params: Optional[dict] = None,
    custom_note: str = 'default'
) -> dict:
    params_str = params_to_str(strategy_params)
    interval = '1h'
    strategy_name = 'turtle'
    strategy_params = {}
    rate = 0

    vt_symbol = comodity_to_vt_symbol(commodity, 'main')
    size = future_basic_data.loc[commodity]['size']
    pricetick = future_basic_data.loc[commodity]['pricetick']
    capital = future_basic_data.loc[commodity]['backtest_capital']
    
    engine = ResearchBacktestingEngine()
    engine.set_parameters(
        vt_symbol=vt_symbol,
        interval=interval,
        start=start,
        end=end,
        rate=rate,
        slippage=pricetick,
        size=size,
        pricetick=pricetick,
        capital=capital
    )
    engine.add_strategy(TurtleBStrategy, strategy_params)

    # run backtest
    engine.load_data()
    engine.run_backtesting()

    trades = engine.get_all_trades()
    last_trade = trades[-1]
    if last_trade.offset == Offset.OPEN:
        engine.trades.pop(last_trade.vt_tradeid)
        
    if trade_output:
        fn = f'{commodity}-{params_str}-{custom_note}-trades.csv'
        trade_df = vt_trade_to_df(engine.get_all_trades())
        trade_df = trade_zh_to_en(trade_df)
        trade_df.to_csv(get_output_path(fn))

    pnl_df = engine.calculate_result()
    day_res = engine.calculate_statistics()
    
    if curve_output:
        img_name = f'{commodity}-{params_str}-{custom_note}-pnl-curve.jpg'
        fig = plt.figure(figsize=(16, 10))
        ax = fig.add_subplot(1, 1, 1)
        ax.set_title(f'{commodity}-{params_str}-{custom_note} Balance Curve')
        engine.daily_df['balance'].plot(legend=True, ax=ax)
        fig.savefig(get_output_path(img_name, 'pnl_curves'))

    trades = engine.trades
    trade_res = exhaust_trade_result(trades, size, rate, pricetick, capital, show_long_short_condition=False)

    d = {
        'commodity': commodity,
        'start_date': day_res['start_date'],
        'end_date': day_res['end_date'],
        'capital': day_res['capital'],
        'day_end_balance': day_res['end_balance'],
        'trade_end_balance': trade_res['end_balance'],
        'day_max_drawdown': day_res['max_drawdown'],
        'trade_max_drawdown': trade_res['max_drawdown'],
        'day_max_ddpercent': day_res['max_ddpercent'],
        'trade_max_ddpercent': trade_res['max_ddpercent'],
        'net_pnl': day_res['total_net_pnl'],
        'commission': day_res['total_commission']
        'slippage': day_res['total_slippage']
        'total_return': day_res['total_return'],
        'annual_return': day_res['annual_return'],
        'sharpe_ratio': day_res['sharpe_ratio'],
        'return_drawdown_ratio': day_res['return_drawdown_ratio'],
        'trade_count': trade_res['trade_count'],
        'daily_trade_count': day_res['daily_trade_count'],
        'winning_rate': trade_res['winning_rate'],
        'win_loss_pnl_ratio': trade_res['win_loss_pnl_ratio']
    }
    
    print(f"{commodity}-回测完成")
    return d

#### 多品种回测测试

In [ ]:
commodities = [
    "cu", "al", "zn"
]

# commodities = [
#     "cu", "al", "zn", "pb", "ni", "sn", "au", "ag", "rb", "hc", "bu", "ru", "sp",
#     "m", "y", "a", "b", "p", "c", "cs", "jd", "l", "v", "pp", "j", "jm", "i",
#     "SR", "CF", "ZC", "FG", "TA", "MA", "OI", "RM", "SF", "SM"
# ]
params = {}
note_str = 'default'

res_list = []
columns = []
for commodity in commodities:
    start = get_hot_start(commodity)
    end = datetime(2019, 12, 1)
    
    res = run_research_backtest(commodity, start, end, strategy_params=params)
    res_list.append(res)
    columns = list(res.keys())

params = params_to_str(params)
file_name = f'research_bt_{params}_{note_str}.csv'
df = pd.DataFrame(res_list, columns=columns)
df.to_csv(get_output_path(file_name, 'multi_backtest'), index=False)
df

#### 单品种回测测试

In [ ]:
commodity = 'FG'
start = get_hot_start(commodity)
end = datetime(2019, 12, 1)

res_dict = run_research_backtest(commodity, start, end, trade_output=True, curve_output=True)
res_dict

#### 分析单品种成交记录

In [ ]:
filename = 'FG-trades.csv'
trades = pd.read_csv(get_output_path(filename))
cum_pnl = 0
for idx, trade in trades.iterrows():
    print(f"{trade.vt_symbol} \t {trade.datetime} \t {trade.direction} \t {trade.price}")
    if trade.offset == "open":
        entry = trade.price
        volume = 1 if trade.direction == "long" else -1
    
    if trade.offset == "close":
        pnl = (trade.price - entry) * volume
        cum_pnl += pnl
        print(f"Trade Pnl: {pnl} \t Cum Pnl: {cum_pnl}")
        print("=" * 60)
        print("\n")

#### 统计多品种回测结果

In [ ]:
folder = 'multi_backtest'
file_path = get_output_path('research_backtest_result.csv', folder)

df = pd.read_csv(file_path)

balance_not_same = df['day_end_balance'].map(lambda x: round(x)) != df['trade_end_balance'].map(lambda x: round(x))
if len(df[balance_not_same]) > 0:
    print(df[balance_not_same])

stats_items = ['day_max_ddpercent', 'total_return', 'annual_return', 'sharpe_ratio', 'return_drawdown_ratio',
               'trade_count', 'winning_rate', 'win_loss_pnl_ratio']
df[stats_items].describe()

win_count = sum(df['annual_return'] > 0)
loss_count = sum(df['annual_return'] <= 0)
winning_rate = win_count / df.count()
annual_return_mean = df['annual_return'].mean()
sharpe_mean = df['sharpe_ratio'].mean()
win_mean = df['winning_rate'].mean
win_to_loss = df['win_loss_pnl_ratio'].mean()

win_count, lose_count